In [ ]:
%matplotlib notebook
import mne
import matplotlib.pyplot as plt

fname = "oddball_example_small-fif.gz"

# Read in raw data; raw objects

In [ ]:
raw = mne.io.read_raw_fif(fname)

MNE is *object oriented*. Objects have corresponding methods. Check which by typing `raw.` and pressing TAB:

In [ ]:
raw.

    raw.resample
    raw.filter
    raw.drop_channels
    ...

Can we do further preprocessing?..

In [ ]:
raw.filter(1, 20)

Aha! By default, MNE does not store raw and epochs objects in memory.

In [ ]:
raw = mne.io.read_raw_fif(fname, preload=True)

In [ ]:
raw.filter(1, 20)

Inspecting raw data ...

In [ ]:
raw.plot();

There are many eog artefacts. We will use ICA to correct these. For this, we create an ICA object
and use its `.fit` method on a filtered copy of the raw data:

In [ ]:
ica = mne.preprocessing.ICA(n_components=20, random_state=0)

In [ ]:
ica.fit(raw.copy().filter(8, 35))

In [ ]:
ica.plot_components(outlines="skirt");

We store "bad" components in the ica object.

In [ ]:
ica.exclude = [1, 10, 14, 17, 18, 19]

We could also use one of the automatic algorithms ...

In [ ]:
bad_idx, scores = ica.find_bads_eog(raw, 'SO2', threshold=2)
print(bad_idx)

Let's compare raw and corrected data ...

In [ ]:
raw.plot();

In [ ]:
ica.apply(raw.copy(), exclude=ica.exclude).plot();

## Epochs

For epoching the data, we need event markers. Usually, these are stored in the `raw` object;
in MNE, in a stimulus channel.

In [ ]:
events = mne.find_events(raw)

`events` is simply an array (time in samples, zero, trigger);

In [ ]:
events

... which we can visualize:

In [ ]:
mne.viz.plot_events(events[:100]);

For creating an `mne.Epochs` object, we require, in addition to the `raw` object and the `events` array, a dictionary of the intended condition names and the corresponding trigger numbers.

In [ ]:
event_ids = {"standard/stimulus": 200, "target/stimulus": 100}

epochs = mne.Epochs(raw, events, event_id=event_ids)

In [ ]:
epochs.plot();

(changing to the inline backend now to speed things up.)

In [ ]:
%matplotlib inline

In [ ]:
epochs = ica.apply(epochs, exclude=ica.exclude)

Of course ...

In [ ]:
epochs = mne.Epochs(raw, events, event_id=event_ids, preload=True)
epochs = ica.apply(epochs, exclude=ica.exclude)

The `mne.Epochs` constructor has a number of options, such as time window lengths and rejection thresholds.
Investigate them on your own.

`Epochs` objects also have various methods, different from `raw` objects - e.g., for baselining.

In [ ]:
epochs.apply_baseline((None, 0))

... and many more ...

In [ ]:
epochs.

To subselect only a sample of epochs, a dict-like access mode is available.

In [ ]:
epochs["target"]

Observe how tags selected by forward slashes - "/" - work.

In [ ]:
epochs["stimulus"]

How does the epoched activity look like?

In [ ]:
epochs["target"].plot_image(picks=[13]);

To ensure we have as many Oddball as Standard trials, we can run ...

In [ ]:
epochs.equalize_event_counts(event_ids)
epochs

We can write the Epochs object to disk so we don't have to repeat the
preprocessing later ...

In [ ]:
epochs.save("oddball-epo.fif")  # remember, the data has been cleaned of bad ICs

In [ ]:
epochs_for_tfr = mne.Epochs(raw, events, event_id=event_ids,
                            tmin=-.5, tmax=1.5, preload=True)  # need longer data segment
epochs_for_tfr = ica.apply(epochs_for_tfr, exclude=ica.exclude)
epochs_for_tfr.equalize_event_counts(event_ids);  # to speed up things
epochs_for_tfr.save("oddball-long-epo.fif")